In [ ]:
import numpy as np
import scipy as sp
from scipy.stats import gamma
import pandas as pd
import matplotlib.pyplot as plt
import math

Question 1: Nous cherchons une équation pour la loi à posteriori $\pi(\lambda|X=x)$ ainsi que f(x). L'énnoncé du travail dicte que la plausabilité de cette loi peut se trouver de cette façon: 
$$\pi(\lambda|X=x)=\frac{1}{f(x)}\frac{\lambda^{\alpha-1}e^{-\beta\lambda}B^{\alpha}}{\Gamma(\alpha)}\Pi_{x_i=1}^{T} \lambda e^{-\lambda x_i}$$
F(x) correspond à la constante de normalisation nécéssaire. Autrement dit, la probabilité de trouver une valeur à cette équation sur le domaine des $\lambda$ doit être de 1, il est donc possible de réécrire: 
$$1=\int_0^\infty\bigg[\frac{1}{f(x)}\frac{\lambda^{\alpha-1}e^{-\beta\lambda}B^{\alpha}}{\Gamma(\alpha)}\Pi_{x_i=1}^{T} \lambda e^{-\lambda x_i}\bigg]d\lambda$$

$$f(x)=\frac{B^{\alpha}}{\Gamma(\alpha)}\int_0^\infty\bigg[\lambda^{\alpha-1}e^{-\beta\lambda} \lambda^T \Pi_{i=1}^T e^{-\lambda x_i} \bigg]d\lambda$$
$$f(x)=\frac{B^{\alpha}}{\Gamma(\alpha)}\int_0^\infty\bigg[\lambda^{\alpha-1}e^{-\beta\lambda} \lambda^T e^{-\lambda\cdot \sum_{x_i=1}^{\infty}x_i}\bigg]d\lambda$$

Pour simplifier les calculs, posons, $\sum_{x_i=1}^{\infty}x_i=C_1$,

$$f(x)=\frac{B^{\alpha}}{\Gamma(\alpha)}\int_0^\infty \lambda^{\alpha-1+T}e^{-\beta\lambda-C_1\lambda}d\lambda$$

En intégrant avec Wolfram Alpha, l'équation devient
$$f(x)=\frac{B^{\alpha}}{\Gamma(\alpha)}\bigg[ \Gamma(T+\alpha)(\Beta+\sum_{x_i=1}^{\infty}x_i)^{-\alpha-T}\bigg]$$












Question 2:
$$\lambda_0=\int_0^\infty \pi(\lambda)d\lambda=\int_0^\infty \lambda^{\alpha-1}e^{-\beta \lambda}\beta^\lambda d\lambda$$

À CONTINUER




Question 3: Pour lire les trois fichiers, il est possible d'utiliser le module pandas. 

In [ ]:

data0=pd.read_csv(f"Tp1/data/activite_temps_0.txt", sep=" ", header=None)
data1=pd.read_csv(f"Tp1/data/activite_temps_1.txt", sep=" ", header=None)
data2=pd.read_csv(f"Tp1/data/activite_temps_2.txt", sep=" ", header=None)

Question 4:

$$f(x)=\int_0^\infty\frac{B^{\alpha}\lambda^{\alpha-1}e^{-\beta\lambda}}{\Gamma(\alpha)} \lambda^T e^{-\lambda\cdot \sum_{x_i=1}^{\infty}x_i}d\lambda$$

$$\hat{\lambda}=\int_0^{200}=\lambda\frac{1}{f(x)}\frac{B^{\alpha}\lambda^{\alpha-1}e^{-\beta\lambda}}{\Gamma(\alpha)} \lambda^T e^{-\lambda\cdot \sum_{x_i=1}^{\infty}x_i}d\lambda$$
Pour se simplifier la tâche lors de la construction de ces équations dans python, nous pouvons y aller en plusieurs morceaux. Premièrement, le terme $\frac{\lambda^{\alpha-1}e^{-\Beta\lambda}B^{\alpha}}{\Gamma(\alpha)}$ peut être représenté comme un fonction gamma avec les paramètres $\alpha=2$,$\beta=\frac{1}{4}$ grâce au module scipy.

In [ ]:
alpha=2
beta=1/4 
dataset=data0
sum_x=float(pd.DataFrame.sum(dataset)) #la sommation des valeurs de X1,X2,X3... est faite 1 seul fois ici




################### Méthode de simpson
def simpson(f, a, b, N):        #Fonction qui effectue le calcul de l'intégral
    h = (b - a) / N             #Le pas utilisé (dx)
    x = np.linspace(a, b, N) 
    s1=0.0
    for k in range(1, N, 2):
        s1 += f(a+k*h)
    s2=0.0
    for k in range(2, N, 2):
        s2 += f(a+k*h)
    return((f(a)+f(b)+4.0*s1+2.0*s1)*h/3.0)

def f(x):
    return(gamma.pdf(x, alpha, loc=0, scale=1/beta)*(x**len(dataset))*math.exp(-x*sum_x))

fx=simpson(f, 0, 200, 10000)

def lambda_ideal(x):
    return(x*(1/fx)*gamma.pdf(x, alpha, loc=0, scale=1/beta)*x**len(dataset)*math.exp(-x*sum_x))

lambda_hat=simpson(lambda_ideal, 0, 200, 10000)

##############################


C:\Users\merciegu\AppData\Local\Temp\ipykernel_15520\2956582109.py:27: RuntimeWarning: overflow encountered in double_scalars
  return(x*(1/fx)*gamma.pdf(x, alpha, loc=0, scale=1/beta)*x**len(dataset)*math.exp(-x*sum_x))
C:\Users\merciegu\AppData\Local\Temp\ipykernel_15520\2956582109.py:27: RuntimeWarning: invalid value encountered in double_scalars
  return(x*(1/fx)*gamma.pdf(x, alpha, loc=0, scale=1/beta)*x**len(dataset)*math.exp(-x*sum_x))


nan


C:\Users\merciegu\AppData\Local\Temp\ipykernel_15520\2956582109.py:27: RuntimeWarning: overflow encountered in multiply
  return(x*(1/fx)*gamma.pdf(x, alpha, loc=0, scale=1/beta)*x**len(dataset)*math.exp(-x*sum_x))


Pour la méthode de Romberg, il faut premièrement définir un calcul de l'intégral avec la méthode du trapèze, puis par la suite itérer en augmentant le nombre de tranche jusqu'à ce que l'erreur sur calculs soit en dessous de notre critère. Deux équations sont nécéssaires soit celle de l'erreur: $c_m h_i^{2m}=\frac{1}{4^m-1}(R_{i,m}-R_{i-1,m})+O(h_i^{2m+2})$. Celle-ci peut être calcu

In [72]:
def trapèze(f,a, b, N):
    h=(b-a)/N
    s=0.5*f(a)+0.5*f(b)
    for k in range(1,N):
        s += f(a+k*h)
    return(h*s)

def romberg(f, a, b, N, epsilon):
    count=0
    flag = False
    R = np.zeros((N, N))
    for i in range(30, N):
        count += 1
        R[i, 0] = trapèze(f, a, b, 2**i)
        for m in range(0, i):
            R[i, m+1] = (4**(m+1) * R[i, m] - R[i-1, m]) / (4**(m+1) - 1)
            if (R[i, m]-R[i-1, m])/(4**i-1) < epsilon and count > 150:
                flag = True
                print("limite erreur atteinte")
                break
        if flag == True:
            break
        print(R) # résultats intermédiaires
    return R
romberg(f, 0, 200, 100, 1E-100)





















KeyboardInterrupt: 

Question 5: Erreur sur la mesure
L'erreur engendré par chacune des méthodes d'intégration peut être calculé avec l'équation 5.30 du Computational Physics de Newman: $\epsilon=\frac{1}{3}[I_i-I_{i-1}]$, où $I_i$ est la valeur calculé de l'intégral pour un nombre de tranche i. 